In [1]:
import numpy as np
import pandas as pd
import queue
import matplotlib.pyplot as plt

import sys
backtest_dir = 'C://backtest/backtest/'
if backtest_dir not in sys.path:
    sys.path.insert(0, backtest_dir)
    
from Backtest import *
from BayesianOptimization import *
from MACD_RSIStrategy import MACD_RSIStrategy
from Backtest.open_json_gz_files import open_json_gz_files
from Backtest.generate_bars import generate_bars

In [2]:
def run_backtest(config, trading_data, ohlc_data, short_window, delta_window, window, s, b):
    short_window = int(short_window)
    long_window = short_window + int(delta_window)
    window = int(window)
    s = int(s)
    b = int(b)
    config['title'] = "MACD_RSIStrategy" + "_" + str(short_window) + "_" + str(long_window) + "_" + str(window) + "_" + str(s) + "_" + str(b)
    print("---------------------------------")
    print(config['title'])
    print("---------------------------------")

    events_queue = queue.Queue()

    data_handler = OHLCDataHandler(
        config, events_queue,
        trading_data = trading_data, ohlc_data = ohlc_data
    )
    strategy = MACD_RSIStrategy(config, events_queue, data_handler,
                            short_window=short_window, long_window=long_window,
                            window=window, s=s, b=b)

    backtest = Backtest(config, events_queue, strategy,
                        data_handler= data_handler)

    results = backtest.start_trading()
    
    return (results['cum_returns'][-1] - 1)

## Setting

In [3]:
config = {
    "csv_dir": "C:/backtest/Binance",
    "out_dir": "C:/backtest/results/MACD_RSIStrategy",
    "title": "MACD_RSIStrategy",
    "is_plot": False,
    "save_plot": False,
    "save_tradelog": False,
    "start_date": pd.Timestamp("2017-07-01T00:0:00", freq = "60" + "T"),    # str(freq) + "T"
    "end_date": pd.Timestamp("2018-04-01T00:00:00", freq = "60" + "T"),
    "equity": 1.0,
    "freq": 60,      # min
    "commission_ratio": 0.001,
    "suggested_quantity": None,     # None or a value
    "max_quantity": None,           # None or a value, Maximum purchase quantity
    "min_quantity": None,           # None or a value, Minimum purchase quantity
    "min_handheld_cash": None,      # None or a value, Minimum handheld funds
    "exchange": "Binance",
    "tickers": ['BTCUSDT']
}

In [4]:
# trading_data = {}
# for ticker in config['tickers']:
#     # trading_data[ticker] = open_gz_files(config['csv_dir'], ticker)
#     trading_data[ticker] = pd.read_hdf(config['csv_dir'] + '\\' + ticker + '.h5', key=ticker)

ohlc_data = {}
for ticker in config['tickers']:
    # ohlc_data[ticker] = generate_bars(trading_data, ticker, config['freq'])
    ohlc_data[ticker] = pd.read_hdf(config['csv_dir'] + '\\' + ticker +'_OHLC_60min.h5', key=ticker)

trading_data = None

In [5]:
gp_params = {"alpha": 1e-5}

## Acquisition Function "Upper Confidence Bound"

* Prefer exploitation (kappa=1.0)

* Prefer exploration (kappa=10)


## Acquisition Function "Expected Improvement"

* Prefer exploitation (xi=1e-4)
* Prefer exploration (xi=0.1)

In [6]:
BO = BayesianOptimization(
    run_backtest,
    {'short_window': (1, 120),
    'delta_window': (1, 120),
    'window': (1, 240),
   's': (55, 85),
  'b': (15, 45)},
    is_int = [1, 1, 1, 1, 1], 
    invariant = {
        'config': config,
        'trading_data': trading_data,
        'ohlc_data': ohlc_data
    },
    random_state = 1
)
# BO.explore({
#     'short_window': np.arange(1, 120, 20),
#     'delta_window': np.arange(1, 120, 20)
#     },
#     eager=True)
BO.maximize(init_points=80, n_iter=120, acq="ei", xi=0.01, **gp_params)

Initialization
---------------------------------------------------------------------------------------------------
 Step |   Time |      Value |         b |   delta_window |         s |   short_window |    window | 
---------------------------------
MACD_RSIStrategy_38_126_61_65_39
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 1.5296220415
Sortino Ratio: nan
Max Drawdown: 0.1240292208
Max Drawdown Duration: 198100
Total Returns: 0.0150112969
Annualized Returns: 0.0353301883
Compound Annual Growth Rate: 0.0356897569
---------------------------------
Trades: 1
Trade Winning: 100.00%
Average Trade: 1.70%
Average Win: 1.70%
Average Loss: nan%
Best Trade: 1.70%
Worst Trade: 1.70%
Worst Trade Date: 0 days 11:00:00
Avg Day

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: nan
Sortino Ratio: nan
Max Drawdown: 0.0000000000
Max Drawdown Duration: 0
Total Returns: 0.0000000000
Annualized Returns: 0.0000000000
Compound Annual Growth Rate: 0.0000000000
---------------------------------
Trades: 0
Trade Winning: nan%
Average Trade: nan%
Average Win: nan%
Average Loss: nan%
Best Trade: nan%
Worst Trade: nan%
Worst Trade Date: 0
Avg Days in Trade: 0
---------------------------------
   10 | 00m07s |    0.00000 | 22.000000 | 24.000000 | 66.000000 | 17.000000 | 104.000000 | 
---------------------------------
MACD_RSIStrategy_2_103_99_58_18
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: nan
Sortino

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: -1.7643882038
Sortino Ratio: -0.3752792693
Max Drawdown: 0.0847132223
Max Drawdown Duration: 85900
Total Returns: -0.0622923089
Annualized Returns: -0.1466095181
Compound Annual Growth Rate: -0.1404745094
---------------------------------
Trades: 3
Trade Winning: 33.33%
Average Trade: -1.91%
Average Win: 0.48%
Average Loss: -3.10%
Best Trade: 0.48%
Worst Trade: -4.32%
Worst Trade Date: 0 days 21:00:00
Avg Days in Trade: 0 days 10:00:00
---------------------------------
   20 | 00m05s |   -0.06229 | 40.000000 | 11.000000 | 55.000000 | 19.000000 | 72.000000 | 
---------------------------------
MACD_RSIStrategy_85_109_232_84_21
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest com

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: nan
Sortino Ratio: nan
Max Drawdown: 0.0000000000
Max Drawdown Duration: 0
Total Returns: 0.0000000000
Annualized Returns: 0.0000000000
Compound Annual Growth Rate: 0.0000000000
---------------------------------
Trades: 0
Trade Winning: nan%
Average Trade: nan%
Average Win: nan%
Average Loss: nan%
Best Trade: nan%
Worst Trade: nan%
Worst Trade Date: 0
Avg Days in Trade: 0
---------------------------------
   29 | 00m12s |    0.00000 | 37.000000 | 75.000000 | 58.000000 | 88.000000 | 119.000000 | 
---------------------------------
MACD_RSIStrategy_88_151_177_55_38
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: nan
Sorti

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: -0.1705702807
Sortino Ratio: -0.1265204062
Max Drawdown: 0.2158144633
Max Drawdown Duration: 334500
Total Returns: -0.0275578250
Annualized Returns: -0.0648593624
Compound Annual Growth Rate: -0.0636536320
---------------------------------
Trades: 20
Trade Winning: 55.00%
Average Trade: 0.12%
Average Win: 2.39%
Average Loss: -2.65%
Best Trade: 10.90%
Worst Trade: -7.05%
Worst Trade Date: 0 days 07:00:00
Avg Days in Trade: 0 days 05:51:00
---------------------------------
   39 | 00m05s |   -0.02756 | 42.000000 | 4.000000 | 74.000000 | 8.000000 | 97.000000 | 
---------------------------------
MACD_RSIStrategy_64_65_184_79_19
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest comp

...

---------------------------------
MACD_RSIStrategy_39_146_140_74_41
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: nan
Sortino Ratio: nan
Max Drawdown: 0.0000000000
Max Drawdown Duration: 0
Total Returns: 0.0000000000
Annualized Returns: 0.0000000000
Compound Annual Growth Rate: 0.0000000000
---------------------------------
Trades: 0
Trade Winning: nan%
Average Trade: nan%
Average Win: nan%
Average Loss: nan%
Best Trade: nan%
Worst Trade: nan%
Worst Trade Date: 0
Avg Days in Trade: 0
---------------------------------
  174 | 00m37s |    0.00000 | 41.000000 | 107.000000 | 74.000000 | 39.000000 | 140.000000 | 
---------------------------------
MACD_RSIStrategy_69_125_160_63_43
---------------------------------
Data Ti

---------------------------------
MACD_RSIStrategy_62_126_192_55_28
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: nan
Sortino Ratio: nan
Max Drawdown: 0.0000000000
Max Drawdown Duration: 0
Total Returns: 0.0000000000
Annualized Returns: 0.0000000000
Compound Annual Growth Rate: 0.0000000000
---------------------------------
Trades: 0
Trade Winning: nan%
Average Trade: nan%
Average Win: nan%
Average Loss: nan%
Best Trade: nan%
Worst Trade: nan%
Worst Trade Date: 0
Avg Days in Trade: 0
---------------------------------
  183 | 00m39s |    0.00000 | 28.000000 | 64.000000 | 55.000000 | 62.000000 | 192.000000 | 
---------------------------------
MACD_RSIStrategy_34_120_240_85_45
---------------------------------
Data Tim

---------------------------------
MACD_RSIStrategy_76_124_157_69_41
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: nan
Sortino Ratio: nan
Max Drawdown: 0.0000000000
Max Drawdown Duration: 0
Total Returns: 0.0000000000
Annualized Returns: 0.0000000000
Compound Annual Growth Rate: 0.0000000000
---------------------------------
Trades: 0
Trade Winning: nan%
Average Trade: nan%
Average Win: nan%
Average Loss: nan%
Best Trade: nan%
Worst Trade: nan%
Worst Trade Date: 0
Avg Days in Trade: 0
---------------------------------
  192 | 00m36s |    0.00000 | 41.000000 | 48.000000 | 69.000000 | 76.000000 | 157.000000 | 


C:\Anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-3.76383592e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 56, 'nit': 5, 'warnflag': 2}
  " state: %s" % convergence_dict)


---------------------------------
MACD_RSIStrategy_59_156_101_79_16
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: nan
Sortino Ratio: nan
Max Drawdown: 0.0000000000
Max Drawdown Duration: 0
Total Returns: 0.0000000000
Annualized Returns: 0.0000000000
Compound Annual Growth Rate: 0.0000000000
---------------------------------
Trades: 0
Trade Winning: nan%
Average Trade: nan%
Average Win: nan%
Average Loss: nan%
Best Trade: nan%
Worst Trade: nan%
Worst Trade Date: 0
Avg Days in Trade: 0
---------------------------------
  193 | 00m43s |    0.00000 | 16.000000 | 97.000000 | 79.000000 | 59.000000 | 101.000000 | 
---------------------------------
MACD_RSIStrategy_75_95_164_64_23
---------------------------------
Data Time

In [7]:
print(BO.res['max'])

{'max_val': 0.38291184981291937, 'max_params': {'short_window': 44.0, 'delta_window': 82.0, 'window': 10.0, 's': 72.0, 'b': 29.0}}


In [8]:
BO.maximize(init_points=0, n_iter=120, acq="ei", xi=0.0001, **gp_params)

Bayesian Optimization
---------------------------------------------------------------------------------------------------
 Step |   Time |      Value |         b |   delta_window |         s |   short_window |    window | 
---------------------------------
MACD_RSIStrategy_70_154_135_56_19
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: nan
Sortino Ratio: nan
Max Drawdown: 0.0000000000
Max Drawdown Duration: 0
Total Returns: 0.0000000000
Annualized Returns: 0.0000000000
Compound Annual Growth Rate: 0.0000000000
---------------------------------
Trades: 0
Trade Winning: nan%
Average Trade: nan%
Average Win: nan%
Average Loss: nan%
Best Trade: nan%
Worst Trade: nan%
Worst Trade Date: 0
Avg Days in Trade: 0
-------------

---------------------------------
MACD_RSIStrategy_1_70_13_74_29
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 0.1876486756
Sortino Ratio: 0.0898068328
Max Drawdown: 0.1520565778
Max Drawdown Duration: 272200
Total Returns: 0.0037708334
Annualized Returns: 0.0088749330
Compound Annual Growth Rate: 0.0088975923
---------------------------------
Trades: 14
Trade Winning: 35.71%
Average Trade: 0.30%
Average Win: 4.52%
Average Loss: -2.04%
Best Trade: 10.64%
Worst Trade: -5.79%
Worst Trade Date: 0 days 09:00:00
Avg Days in Trade: 0 days 09:08:34
---------------------------------
  210 | 00m37s |    0.00377 | 29.000000 | 69.000000 | 74.000000 | 1.000000 | 13.000000 | 
---------------------------------
MACD_RSIStrategy_52

C:\Anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([0.00011112]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 53, 'nit': 3, 'warnflag': 2}
  " state: %s" % convergence_dict)


---------------------------------
MACD_RSIStrategy_54_100_68_74_17
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: nan
Sortino Ratio: nan
Max Drawdown: 0.0000000000
Max Drawdown Duration: 0
Total Returns: 0.0000000000
Annualized Returns: 0.0000000000
Compound Annual Growth Rate: 0.0000000000
---------------------------------
Trades: 0
Trade Winning: nan%
Average Trade: nan%
Average Win: nan%
Average Loss: nan%
Best Trade: nan%
Worst Trade: nan%
Worst Trade Date: 0
Avg Days in Trade: 0
---------------------------------
  212 | 01m01s |    0.00000 | 17.000000 | 46.000000 | 74.000000 | 54.000000 | 68.000000 | 
---------------------------------
MACD_RSIStrategy_30_105_240_55_45
---------------------------------
Data Time 

---------------------------------
MACD_RSIStrategy_18_86_36_71_20
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: nan
Sortino Ratio: nan
Max Drawdown: 0.0000000000
Max Drawdown Duration: 0
Total Returns: 0.0000000000
Annualized Returns: 0.0000000000
Compound Annual Growth Rate: 0.0000000000
---------------------------------
Trades: 0
Trade Winning: nan%
Average Trade: nan%
Average Win: nan%
Average Loss: nan%
Best Trade: nan%
Worst Trade: nan%
Worst Trade Date: 0
Avg Days in Trade: 0
---------------------------------
  221 | 01m01s |    0.00000 | 20.000000 | 68.000000 | 71.000000 | 18.000000 | 36.000000 | 
---------------------------------
MACD_RSIStrategy_120_240_209_55_45
---------------------------------
Data Time 

...

---------------------------------
MACD_RSIStrategy_1_121_121_55_45
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 2.3285170934
Sortino Ratio: nan
Max Drawdown: 0.0312500000
Max Drawdown Duration: 26700
Total Returns: 0.0830616287
Annualized Returns: 0.1954916356
Compound Annual Growth Rate: 0.2065873745
---------------------------------
Trades: 2
Trade Winning: 50.00%
Average Trade: 4.42%
Average Win: 9.83%
Average Loss: -0.99%
Best Trade: 9.83%
Worst Trade: -0.99%
Worst Trade Date: 0 days 01:00:00
Avg Days in Trade: 0 days 23:30:00
---------------------------------
  289 | 01m56s |    0.08306 | 45.000000 | 120.000000 | 55.000000 | 1.000000 | 121.000000 | 
---------------------------------
MACD_RSIStrategy_5_15_36_68

C:\Anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-3.63774411e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 56, 'nit': 4, 'warnflag': 2}
  " state: %s" % convergence_dict)


---------------------------------
MACD_RSIStrategy_46_84_99_74_24
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: nan
Sortino Ratio: nan
Max Drawdown: 0.0000000000
Max Drawdown Duration: 0
Total Returns: 0.0000000000
Annualized Returns: 0.0000000000
Compound Annual Growth Rate: 0.0000000000
---------------------------------
Trades: 0
Trade Winning: nan%
Average Trade: nan%
Average Win: nan%
Average Loss: nan%
Best Trade: nan%
Worst Trade: nan%
Worst Trade Date: 0
Avg Days in Trade: 0
---------------------------------
  294 | 00m30s |    0.00000 | 24.000000 | 38.000000 | 74.000000 | 46.000000 | 99.000000 | 
---------------------------------
MACD_RSIStrategy_28_92_1_69_34
---------------------------------
Data Time Inte

---------------------------------
MACD_RSIStrategy_58_92_16_62_29
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: -1.5476911316
Sortino Ratio: -0.8738081112
Max Drawdown: 0.3043248488
Max Drawdown Duration: 225700
Total Returns: -0.2290998148
Annualized Returns: -0.5392032180
Compound Annual Growth Rate: -0.4579468686
---------------------------------
Trades: 18
Trade Winning: 44.44%
Average Trade: -1.14%
Average Win: 2.56%
Average Loss: -4.10%
Best Trade: 5.85%
Worst Trade: -10.50%
Worst Trade Date: 1 days 06:00:00
Avg Days in Trade: 0 days 18:53:20
---------------------------------
  303 | 02m32s |   -0.22910 | 29.000000 | 34.000000 | 62.000000 | 58.000000 | 16.000000 | 
---------------------------------
MACD_RSIStr

---------------------------------
MACD_RSIStrategy_57_97_66_85_45
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: -0.4398393559
Sortino Ratio: -0.1980974222
Max Drawdown: 0.1927334746
Max Drawdown Duration: 145300
Total Returns: -0.0478863189
Annualized Returns: -0.1127039639
Compound Annual Growth Rate: -0.1090721377
---------------------------------
Trades: 13
Trade Winning: 46.15%
Average Trade: -0.09%
Average Win: 3.38%
Average Loss: -3.05%
Best Trade: 10.36%
Worst Trade: -8.42%
Worst Trade Date: 0 days 08:00:00
Avg Days in Trade: 0 days 08:27:42
---------------------------------
  312 | 02m49s |   -0.04789 | 45.000000 | 40.000000 | 85.000000 | 57.000000 | 66.000000 | 
---------------------------------
MACD_RSIStr

In [9]:
print(BO.res['max'])

{'max_val': 0.7846659801206235, 'max_params': {'short_window': 41.0, 'delta_window': 72.0, 'window': 2.0, 's': 74.0, 'b': 36.0}}


In [10]:
Target = pd.DataFrame({'Parameters': BO.X.tolist(), 'Target': BO.Y})
# Target.to_csv(config['out_dir'] + "/target_ei.csv")
Target.sort_values(by = "Target")

,Parameters,Target
244,"[1.0, 1.0, 37.0, 85.0, 45.0]",-0.786224
117,"[80.0, 112.0, 24.0, 81.0, 31.0]",-0.433841
116,"[120.0, 120.0, 96.0, 85.0, 45.0]",-0.402371
241,"[2.0, 118.0, 22.0, 65.0, 42.0]",-0.402167
146,"[1.0, 77.0, 1.0, 55.0, 45.0]",-0.387437
255,"[83.0, 33.0, 1.0, 78.0, 44.0]",-0.373797
102,"[120.0, 120.0, 1.0, 55.0, 15.0]",-0.367504
99,"[82.0, 1.0, 1.0, 85.0, 15.0]",-0.366117
272,"[87.0, 75.0, 31.0, 74.0, 36.0]",-0.329978
310,"[120.0, 65.0, 1.0, 55.0, 15.0]",-0.327985


## Acquisition Function "Probability of Improvement"

* Prefer exploitation (xi=1e-4)
* Prefer exploration (xi=0.1)